# Hospital Resource and Performance Analytics Using SQL

This notebook explores hospital operations using SQL inside Python.
We will connect multiple CSV datasets (patients, staff, services, and schedules),
load them into an SQLite database, and run SQL queries to analyze:

- Bed capacity and shortages
- Staff attendance and morale
- Patient satisfaction
- Overall service performance



In [22]:
import pandas as pd
import sqlite3
import kagglehub

# Download dataset
path = kagglehub.dataset_download("jaderz/hospital-beds-management")
print("Path to dataset files:", path)


Using Colab cache for faster access to the 'hospital-beds-management' dataset.
Path to dataset files: /kaggle/input/hospital-beds-management


In [23]:
patients = pd.read_csv(f"{path}/patients.csv")
services_weekly = pd.read_csv(f"{path}/services_weekly.csv")
staff = pd.read_csv(f"{path}/staff.csv")
staff_schedule = pd.read_csv(f"{path}/staff_schedule.csv")

patients.head(), services_weekly.head()


(     patient_id               name  age arrival_date departure_date  \
 0  PAT-09484753  Richard Rodriguez   24   2025-03-16     2025-03-22   
 1  PAT-f0644084     Shannon Walker    6   2025-12-13     2025-12-14   
 2  PAT-ac6162e4       Julia Torres   24   2025-06-29     2025-07-05   
 3  PAT-3dda2bb5    Crystal Johnson   32   2025-10-12     2025-10-23   
 4  PAT-08591375        Garrett Lin   25   2025-02-18     2025-02-25   
 
             service  satisfaction  
 0           surgery            61  
 1           surgery            83  
 2  general_medicine            83  
 3         emergency            81  
 4               ICU            76  ,
    week  month           service  available_beds  patients_request  \
 0     1      1         emergency              32                76   
 1     1      1           surgery              45               130   
 2     1      1  general_medicine              37               201   
 3     1      1               ICU              22          

In [24]:
conn = sqlite3.connect('hospital.db')

patients.to_sql('patients', conn, if_exists='replace', index=False)
services_weekly.to_sql('services_weekly', conn, if_exists='replace', index=False)
staff.to_sql('staff', conn, if_exists='replace', index=False)
staff_schedule.to_sql('staff_schedule', conn, if_exists='replace', index=False)


6552

In [25]:
query1 = '''
SELECT service,
       AVG(CAST(patients_admitted AS FLOAT) / available_beds) * 100 AS avg_occupancy_rate
FROM services_weekly
GROUP BY service
ORDER BY avg_occupancy_rate DESC;
'''

pd.read_sql_query(query1, conn)


,service,avg_occupancy_rate
0,emergency,100.000000
1,general_medicine,97.313582
2,surgery,88.186759
3,ICU,84.381561


In [26]:
query2 = '''
SELECT week,
       AVG(CAST(patients_admitted AS FLOAT) / available_beds) * 100 AS avg_utilization
FROM services_weekly
GROUP BY week
ORDER BY week;
'''

pd.read_sql_query(query2, conn)


,week,avg_utilization
0,1,100.000000
1,2,77.187500
2,3,100.000000
3,4,100.000000
4,5,89.772727
5,6,96.428571
6,7,100.000000
7,8,80.637255
8,9,100.000000
9,10,90.384615


In [27]:
query3 = '''
SELECT service,
       SUM(patients_refused) AS total_refused,
       SUM(patients_request) AS total_requested,
       (SUM(CAST(patients_refused AS FLOAT)) / SUM(patients_request)) * 100 AS refusal_rate
FROM services_weekly
GROUP BY service
ORDER BY refusal_rate DESC;
'''

pd.read_sql_query(query3, conn)


,service,total_refused,total_requested,refusal_rate
0,emergency,5008,6193,80.865493
1,general_medicine,1938,4270,45.386417
2,surgery,555,2241,24.765730
3,ICU,141,789,17.870722


In [28]:
query4 = '''
SELECT s.service,
       AVG(sc.present) * 100 AS attendance_rate
FROM staff_schedule sc
JOIN staff s ON sc.staff_id = s.staff_id
GROUP BY s.service
ORDER BY attendance_rate ASC;
'''

pd.read_sql_query(query4, conn)


,service,attendance_rate


In [29]:
query5 = '''
SELECT service,
       ROUND(AVG(staff_morale),1) AS avg_morale,
       ROUND(AVG(patient_satisfaction),1) AS avg_satisfaction
FROM services_weekly
GROUP BY service
ORDER BY avg_morale DESC;
'''

pd.read_sql_query(query5, conn)


,service,avg_morale,avg_satisfaction
0,emergency,73.6,77.9
1,general_medicine,73.1,81.2
2,surgery,72.6,79.3
3,ICU,71.0,81.6


In [30]:
query6 = '''
SELECT event,
       AVG(patients_refused) AS avg_refused,
       AVG(staff_morale) AS avg_morale,
       AVG(patient_satisfaction) AS avg_satisfaction
FROM services_weekly
WHERE event <> 'none'
GROUP BY event;
'''

pd.read_sql_query(query6, conn)


,event,avg_refused,avg_morale,avg_satisfaction
0,donation,29.142857,80.142857,82.714286
1,flu,126.157895,72.894737,78.736842
2,strike,16.636364,53.727273,82.818182


In [31]:
query7 = '''
SELECT service,
       ROUND(AVG(satisfaction),1) AS avg_satisfaction,
       COUNT(*) AS num_patients
FROM patients
GROUP BY service
ORDER BY avg_satisfaction DESC;
'''

pd.read_sql_query(query7, conn)


,service,avg_satisfaction,num_patients
0,surgery,80.3,254
1,ICU,79.9,241
2,emergency,79.5,263
3,general_medicine,78.6,242


In [32]:
query8 = '''
SELECT service,
       AVG(julianday(departure_date) - julianday(arrival_date)) AS avg_stay_days
FROM patients
GROUP BY service;
'''

pd.read_sql_query(query8, conn)


,service,avg_stay_days
0,ICU,7.605809
1,emergency,7.159696
2,general_medicine,6.995868
3,surgery,7.866142


In [33]:
query9 = '''
SELECT s.service,
       ROUND(AVG(sc.present)*100,1) AS attendance_rate,
       ROUND(AVG(CAST(sw.patients_refused AS FLOAT) / sw.patients_request)*100,1) AS refusal_rate
FROM staff_schedule sc
JOIN staff s ON sc.staff_id = s.staff_id
JOIN services_weekly sw ON s.service = sw.service AND sc.week = sw.week
GROUP BY s.service
ORDER BY refusal_rate DESC;
'''

pd.read_sql_query(query9, conn)


,service,attendance_rate,refusal_rate


In [34]:
query10 = '''
SELECT service,
       ROUND(AVG(patient_satisfaction),1) AS satisfaction,
       ROUND(AVG(staff_morale),1) AS morale,
       ROUND(AVG(CAST(patients_admitted AS FLOAT) / available_beds)*100,1) AS occupancy
FROM services_weekly
GROUP BY service
ORDER BY satisfaction DESC, morale DESC;
'''

pd.read_sql_query(query10, conn)


,service,satisfaction,morale,occupancy
0,ICU,81.6,71.0,84.4
1,general_medicine,81.2,73.1,97.3
2,surgery,79.3,72.6,88.2
3,emergency,77.9,73.6,100.0


In [35]:
conn.close()


This notebook demonstrates a complete SQL-based analysis workflow.
It connects multiple data sources, computes key hospital metrics,
and reveals patterns between staff, patients, and operational capacity.
